In [1]:
#Importing required libraries
import selenium
import pandas as pd
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

# 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product name to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
#Connect to web driver
driver=webdriver.Chrome(r"D://chromedriver.exe")  #r converts string to raw string
#If not r, we can use executable_path = "C:/path name"

In [3]:
#Getting the website to driver
driver.get('https://www.amazon.in/')

#When we run this line, automatically the webpage will be opened

In [4]:
#Searching headphones in the search bar and clicking the search button
search_bar=driver.find_element_by_id('twotabsearchtextbox')
search_bar.send_keys("headphones")

driver.find_element_by_id('nav-search-submit-button').click()

# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“. 

In [5]:
#Creating a for loop to append the urls of first 3 pages
URL=[]  #Empty list
for page in range(0,3):
    for i in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-medium']//h2"):
        URL.append(i.find_element_by_xpath(".//a").get_attribute('href'))
    try:
        next_page=driver.find_element_by_xpath("//div[@class='a-text-center']/ul/li[@class='a-last']/a").get_attribute('href')
        driver.get(next_page)
        driver.refresh()   
    except:
        break   
        
#Taking the empty lists
brand_name=[]
product_name=[]
Rating=[]
no_of_ratings=[]
price=[]
returns=[]
delivery=[]
availability=[]
other_details=[]
product_url=[]
     
#We are running a for loop for the url as it changes for every page and we are extracting the product url
for i in URL:
    driver.get(i)
    product_url.append(i)
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the brand name
    try:
        brand_name.append(driver.find_element_by_id("productTitle").text.split(' ')[0])
    except NoSuchElementException as e:
        brand_name.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds   
    
    #Extracting the product name
    try:
        string=' '  #Empty string
        for j in driver.find_element_by_id("productTitle").text.split(' ')[1:]:
            string=string+' '+j  #Extracting the text and appending to empty string
        product_name.append(string)  #Appending the string having the extracted text
    except NoSuchElementException as e:
            product_name.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds     
    
    #Extracting the ratings
    try:
        Rating.append(driver.find_element_by_id("acrPopover").get_attribute('title')) 
    except NoSuchElementException as e:
        Rating.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the number of ratings
    try:
        no_of_ratings.append(driver.find_element_by_id("acrCustomerReviewText").text) 
    except NoSuchElementException as e:
        no_of_ratings.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the price
    try:
        try:
            price.append(driver.find_element_by_id("priceblock_saleprice").text) 
        except:
            try:
                price.append(driver.find_element_by_id("priceblock_dealprice").text)
            except:
                price.append(driver.find_element_by_id("priceblock_ourprice").text)
    except NoSuchElementException as e:
        price.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the Return/Exchange
    try:
        returns.append(driver.find_element_by_xpath("//div[@id='RETURNS_POLICY']").text)
    except NoSuchElementException as e:
        returns.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting Expected Delivery
    try:
        delivery.append(driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b').text)
    except NoSuchElementException as e:
        delivery.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds  
    
    #Extracting the availability of headphones
    try:
        availability.append(driver.find_element_by_xpath("//div[@id='availability']").text)
    except NoSuchElementException as e:    
        availability.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds  
    
    #Extracting the other details available
    try:
        details=[i.text.replace('\n','---') for i in driver.find_element_by_id("productDetails_techSpec_section_1").find_elements_by_xpath(".//tbody")]
        other_details.append(details[0])
    except NoSuchElementException as e:     
        other_details.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds     

In [6]:
#Checking the length of data extracted
print(len(brand_name),len(product_name),len(Rating),len(no_of_ratings),len(price),len(returns),len(delivery),
      len(availability),len(other_details),len(product_url))

66 66 66 66 66 66 66 66 66 66


In [7]:
#Creating a dataframe and checking the data extracted
amazon=pd.DataFrame({'Brand Name':brand_name,'Name of Product':product_name,'Rating':Rating,'No of Ratings':no_of_ratings,
                     'Price':price,'Return/Exchange':returns,'Expected Delivery':delivery,'Availability':availability,
                     'Other details':other_details,'Product URL':product_url})
amazon

,Brand Name,Name of Product,Rating,No of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Other details,Product URL
0,Fire-Boltt,Blast 1400 Over -Ear Bluetooth Wireless Head...,4.1 out of 5 stars,"3,520 ratings","₹ 2,499.00",7 Days Replacement,"Saturday, April 24",In stock.,"Brand Fire-Boltt---Manufacturer Fire-Boltt, Bo...",https://www.amazon.in/gp/slredirect/picassoRed...
1,boAt,Rockerz 450 Bluetooth On-Ear Headphone with ...,4.0 out of 5 stars,"21,368 ratings","₹ 1,499.00",7 Days Replacement,"Saturday, April 24",In stock.,Brand Boat---Manufacturer Imagine Marketing Pv...,https://www.amazon.in/gp/slredirect/picassoRed...
2,pTron,"Boom Ultima 4D Dual Driver, in-Ear Gaming Wi...",3.8 out of 5 stars,"14,804 ratings",₹ 499.00,7 Days Replacement,"Tuesday, April 27",In stock.,Brand PTron---Manufacturer Palred Technology S...,https://www.amazon.in/Boom-Ultima-Headphones-C...
3,Boult,Audio Bass Buds Q2 Over-Ear Wired Lightweigh...,3.9 out of 5 stars,"1,930 ratings",₹ 499.00,7 Days Replacement,"Saturday, April 24",In stock.,"Brand Boult Audio---Manufacturer Exotic Mile, ...",https://www.amazon.in/Boult-Audio-Wired-Lightw...
4,boAt,Bassheads 900 On Ear Wired Headphones(Carbon...,4.2 out of 5 stars,"33,052 ratings",₹ 749.00,7 Days Replacement,"Saturday, April 24",In stock.,Brand Boat---Manufacturer Imagine Marketing Pv...,https://www.amazon.in/Boat-BassHeads-900-Wired...
...,...,...,...,...,...,...,...,...,...,...
61,OneOdio,Adapter-Free Closed Back Over-Ear DJ Stereo ...,4.5 out of 5 stars,"2,806 ratings","₹ 2,089.00",7 Days Replacement Only,"Tuesday, April 27",,Brand OneOdio---Manufacturer OneOdio---Package...,https://www.amazon.in/OneOdio-Adapter-Free-Hea...
62,Zebronics,Zeb-Lark Wireless in Ear Neckband Earphone w...,3.4 out of 5 stars,791 ratings,₹ 798.00,7 Days Replacement,"Saturday, April 24",In stock.,"Brand ZEBRONICS---Manufacturer ZEBRONICS, ZEBR...",https://www.amazon.in/Zebronics-Zeb-Lark-Bluet...
63,Techfire,"Bluetooth Neckband, in-Ear Wireless Earphone...",-,-,₹ 500.00,7 Days Replacement,"Monday, May 3",In stock.,Brand Techfire---Manufacturer Techfire Pvt Lim...,https://www.amazon.in/Techfire-Bluetooth-Sweat...
64,One,Plus Buds (Gray),-,-,"₹ 1,199.00",Not Returnable,April 28 - 30,Only 2 left in stock.,-,https://www.amazon.in/gp/slredirect/picassoRed...


In [8]:
#Saving the data into a csv file
amazon.to_csv("Amazon_Headphones.csv")

#Closing the driver
driver.close()

# 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [9]:
#Connect to web driver
driver=webdriver.Chrome(r"D://chromedriver.exe")  #r converts string to raw string
#If not r, we can use executable_path = "C:/path name"

In [10]:
#Getting the website to driver
url=('https://images.google.com/')
driver.get(url)
#When we run this line, automatically the webpage will be opened

In [11]:
#Defining a function which accepts image names as arguments
def get_images(keywords):
    driver.get(url)
    
    #Searching the keywords in the search bar and clicking the search button
    search_bar=driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
    search_bar.send_keys(keywords)
    button=driver.find_element_by_xpath("//button[@class='Tg7LZd']")
    button.click()
    driver.implicitly_wait(1)  #Making the driver wait for 1 second
    
    #Taking 5 pages for each keyword and scraping the image data
    for page in range(0,5):
        driver.find_element_by_xpath("//a[@class='wXeWr islib nfEiy mM5pbd']").send_keys(Keys.PAGE_DOWN)
        time.sleep(2)   #Delaying the page wait time for 2 seconds
        
        #Iterating data for first 100 results of the keywords
        picture=[image for image in driver.find_elements_by_xpath("//a[@class='wXeWr islib nfEiy mM5pbd']")[:100]]
        image_url=[]   #Empty list
        print(len(picture))  
        
        #Iterating the tags for getting url link text of 100 results
        if(len(picture)==100):  #Checking if the length is equal to 100
            for i in picture:
                try:
                    i.click()
                    image_url.append(i.get_attribute('href'))  #Extracting the link texts
                except NoSuchElementException as e:
                    image_url.append("Link Not Available")  #Exception message if urls are not available
            return image_url
        else:
            print("Image not available")  #Message if length of data is not equal

In [12]:
#Sending the keywords for scraping the urls of images
Fruits_images=get_images('fruits')
Cars_images=get_images('cars')
machine_learning=get_images('Machine Learning')

49
Image not available
49
Image not available
49
Image not available
100
49
Image not available
49
Image not available
49
Image not available
100
49
Image not available
49
Image not available
49
Image not available
49
Image not available
100


In [13]:
#Create a DataFrame for our data extracted
Image_URL=pd.DataFrame({})
Image_URL['Fruits']=Fruits_images
Image_URL['Cars']=Cars_images
Image_URL['Machine Learning']=machine_learning

#Checking the dataframe
Image_URL

,Fruits,Cars,Machine Learning
0,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
1,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
2,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
3,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
4,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
...,...,...,...
95,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
96,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
97,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
98,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...


In [14]:
#Closing the driver
driver.close()

# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV. 

In [15]:
#Connect to web driver
driver=webdriver.Chrome(r"D://chromedriver.exe")  #r converts string to raw string
#If not r, we can use executable_path = "C:/path name"

In [16]:
#Getting the website to driver
driver.get('https://www.flipkart.com/')

#When we run this line, automatically the webpage will be opened

In [17]:
#Searching smartphone in the search bar and clicking the search button
search_bar=driver.find_element_by_xpath("//input[@type='text']")
search_bar.send_keys('oppo')  #The smartphone name to be searched

driver.find_element_by_xpath("//button[@type='submit']").click()  #Clicking the search button

In [18]:
#Specifying the url of the webpage to be scraped
url="https://www.flipkart.com/search?q=oppo&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"
driver.get(url)

In [19]:
#Empty lists to store the scraped data
product_urls=[]
brand_name=[]
phone_name=[]
phone_color=[]
RAM=[]
ROM=[]
p_cam=[]
s_cam=[]
d_size=[]
d_resolution=[]
processor=[]
processor_core=[]
battery=[]
price=[]

#Defining the url link of each smartphone for scraping the specifications details
urls=driver.find_elements_by_xpath("//a[@class='_1fQZEK']")

#Extracting the product urls
for i in urls:
    product_urls.append(i.get_attribute('href'))
    
for i in product_urls:
    driver.get(i)
    
    #Button for expanding the specifications
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
    
    #Extracting the brand name
    try:
        brand=driver.find_element_by_xpath("//span[@class='B_NuCI']").text.split()
        brand_name.append(brand[0])
    except NoSuchElementException as e:
        brand_name.append('-')
        
    #Extracting the smartphone name
    try:
        phone=driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[1]/table/tbody/tr[3]/td[2]")
        phone_name.append(phone.text)
    except NoSuchElementException as e:
        phone_name.append('-')  
        
    #Extracting the phone color
    try:
        color=driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[1]/table/tbody/tr[4]/td[2]")
        phone_color.append(color.text)
    except NoSuchElementException as e:
        phone_color.append('-') 
        
    #Extracting RAM details
    try:
        RAM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[2]/td[2]').text)
    except NoSuchElementException as e:
        RAM.append('-')   
        
    #Extracting ROM details
    try:
        ROM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[1]/td[2]').text)
    except NoSuchElementException as e:
        ROM.append('-')
        
    #Extracting primary camera details
    try:
        p_cam.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[2]/td[2]').text)
    except NoSuchElementException as e:
        p_cam.append('-')
        
    #Extracting secondary camera details
    try:
        s_cam.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[5]/td[2]').text)
    except NoSuchElementException as e:
        s_cam.append('-')
        
    #Extracting display size details
    try:
        d_size.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[1]/td[2]').text)
    except NoSuchElementException as e:
        d_size.append('-')
        
    #Extracting display resolution details
    try:
        d_resolution.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[2]/td[2]').text)
    except NoSuchElementException as e:
        d_resolution.append('-')
        
    #Extracting Processor details
    try:
        processor.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[2]/td[2]').text)
    except NoSuchElementException as e:
        processor.append('-')
        
    #Extracting processor core details
    try:
        processor_core.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[3]/td[2]').text)
    except NoSuchElementException as e: 
        processor_core.append('-')
        
    #Extracting battery details
    try:
        battery.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[10]/table/tbody/tr[1]/td[2]').text)
    except NoSuchElementException as e:
        battery.append('-')
        
    #Extracting price details
    try:
        price.append(driver.find_element_by_xpath('//div[@class="dyC4hf"]/div/div/div').text)
    except NoSuchElementException as e:
        price.append('-')

In [20]:
#Create a DataFrame for our data
flipkart=pd.DataFrame({})
flipkart['Brand']=brand_name
flipkart['Phone Name']=phone_name
flipkart['Color']=phone_color
flipkart['RAM']=RAM
flipkart['Storage (ROM)']=ROM
flipkart['Primary Camera']=p_cam
flipkart['Secondary Camera']=s_cam
flipkart['Display Size']=d_size
flipkart['Display Resolution']=d_resolution
flipkart['Processor']=processor
flipkart['Processor Core']=processor_core
flipkart['Battery Capacity']=battery
flipkart['Price']=price
flipkart['Product URL']=product_urls

#Checking the dataframe
flipkart

,Brand,Phone Name,Color,RAM,Storage (ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Core,Battery Capacity,Price,Product URL
0,OPPO,A12,Blue,4 GB,64 GB,13MP + 2MP,5MP Front Camera,15.8 cm (6.22 inch),1520 x 720 Pixels,MediaTek Helio P35,Octa Core,4230 mAh,"₹11,999",https://www.flipkart.com/oppo-a12-blue-64-gb/p...
1,OPPO,A12,Deep Blue,3 GB,32 GB,13MP + 2MP,5MP Front Camera,15.8 cm (6.22 inch),1520 x 720 Pixels,MediaTek Helio P35,Octa Core,4230 mAh,"₹7,990",https://www.flipkart.com/oppo-a12-deep-blue-32...
2,OPPO,A12,Deep Blue,4 GB,64 GB,13MP + 2MP,5MP Front Camera,15.8 cm (6.22 inch),1520 x 720 Pixels,MediaTek Helio P35,Octa Core,4230 mAh,"₹9,880",https://www.flipkart.com/oppo-a12-deep-blue-64...
3,OPPO,A12,Black,3 GB,32 GB,13MP + 2MP,5MP Front Camera,15.8 cm (6.22 inch),1520 x 720 Pixels,MediaTek Helio P35,Octa Core,4230 mAh,"₹9,999",https://www.flipkart.com/oppo-a12-black-32-gb/...
4,OPPO,A12,Flowing Silver,3 GB,32 GB,13MP + 2MP,5MP Front Camera,15.8 cm (6.22 inch),1520 x 720 Pixels,MediaTek Helio P35,Octa Core,4230 mAh,"₹7,990",https://www.flipkart.com/oppo-a12-flowing-silv...
5,OPPO,A12,Black,4 GB,64 GB,13MP + 2MP,5MP Front Camera,15.8 cm (6.22 inch),1520 x 720 Pixels,MediaTek Helio P35,Octa Core,4230 mAh,"₹11,499",https://www.flipkart.com/oppo-a12-black-64-gb/...
6,OPPO,F19 Pro,Crystal Silver,8 GB,256 GB,48MP + 8MP + 2MP + 2MP,16MP Front Camera,16.33 cm (6.43 inch),2400 x 1080 Pixels,MediaTek Helio P95 (MT6779V),Octa Core,73.2 mm,"₹23,490",https://www.flipkart.com/oppo-f19-pro-crystal-...
7,OPPO,F19 Pro,Fluid Black,8 GB,128 GB,48MP + 8MP + 2MP + 2MP,16MP Front Camera,16.33 cm (6.43 inch),2400 x 1080 Pixels,MediaTek Helio P95 (MT6779V),Octa Core,73.2 mm,"₹21,490",https://www.flipkart.com/oppo-f19-pro-fluid-bl...
8,OPPO,F19 Pro,Fluid Black,8 GB,256 GB,48MP + 8MP + 2MP + 2MP,16MP Front Camera,16.33 cm (6.43 inch),2400 x 1080 Pixels,MediaTek Helio P95 (MT6779V),Octa Core,73.2 mm,"₹23,490",https://www.flipkart.com/oppo-f19-pro-fluid-bl...
9,OPPO,F17 Pro,Magic Blue,8 GB,128 GB,48MP + 8MP + 2MP + 2MP,16MP + 2MP Dual Front Camera,16.33 cm (6.43 inch),2400 x 1080 Pixels,MediaTek Helio P95,Octa Core,73.77 mm,"₹21,490",https://www.flipkart.com/oppo-f17-pro-magic-bl...


In [21]:
#Saving the data into a csv file
flipkart.to_csv("Flipkart_OppoPhones.csv")

#Closing the driver
driver.close()

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [22]:
#Connect to web driver
driver=webdriver.Chrome(r"D://chromedriver.exe")  #r converts string to raw string
#If not r, we can use executable_path = "C:/path name"

In [23]:
#Getting the website to driver
driver.get('https://www.google.com/maps')

#When we run this line, automatically the webpage will be opened

In [24]:
#Creating user input for entering the city manually
city=input("Enter City: ")

#Finding the required element and sending the keys
driver.find_element_by_xpath("//button[@class='searchbox-searchbutton']").send_keys(city)

Enter City: Mumbai


In [25]:
#For Automatically entering the button, we will use Keys library and Keys.Enter function for automatically entering the input
driver.find_element_by_xpath("//button[@class='searchbox-searchbutton']").send_keys(Keys.ENTER)

In [26]:
#For finding the coordinates, we will use driver.current_url function that provides current url automatically
coordinates=driver.current_url
lat=coordinates.split('@')[1].split(',')[0]
long=coordinates.split('@')[1].split(',')[1].split(',')[0]
print('Latitude: ',lat)
print('Longitude: ',long)

Latitude:  19.0821978
Longitude:  72.7410984


In [27]:
#Closing the driver
driver.close()

In [33]:
#Closing the driver
driver.close()

# 7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [34]:
#Connect to web driver
driver=webdriver.Chrome(r"D://chromedriver.exe")  #r converts string to raw string
#If not r, we can use executable_path = "C:/path name"

In [35]:
#Getting the website to driver
driver.get('https://www.digit.in/')

#When we run this line, automatically the webpage will be opened

In [36]:
#Clicking on top 10 option
top10=driver.find_element_by_xpath("//div[@class='menu']/ul/li[4]/a")
top10.click()

#Clicking on best gaming laptops option from top10
best_gl=driver.find_element_by_xpath("//div[@class='listing_container']/ul/li[26]/a")
best_gl.click()

In [37]:
#Specifying the url of the webpage to be scraped
url="https://www.digit.in/top-products/best-gaming-laptops-40.html"
driver.get(url)

In [38]:
#Extracting the tags having the laptop name
name=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")

#Extracting the text from the tags
prod_name=[]  #Empty list

#As we need to scrap data for all the products, we are running a for loop for extracting all data
for i in name:
    prod_name.append(i.text)

#Extracting the tags having the OS type
OS_type=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")

#Extracting the text from the tags
OS=[]  #Empty list

#As we need to scrap data for all the products, we are running a for loop for extracting all data
for i in OS_type:
    OS.append(i.text)

#Extracting the tags having display details
display=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")

#Extracting the text from the tags
display_specs=[]  #Empty list

#As we need to scrap data for all the products, we are running a for loop for extracting all data
for i in display:
    display_specs.append(i.text)

#Extracting the tags having processor details
processor=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")

#Extracting the text from the tags
processor_specs=[]  #Empty list

#As we need to scrap data for all the products, we are running a for loop for extracting all data
for i in processor:
    processor_specs.append(i.text)

#Extracting the tags having memory specs

#List of specification names
memory=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[1]")

#Value of the specifications
memory_specs=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")

#Now we will separate HDD and RAM text from memory specs tags
HDD=[]
RAM=[]  #Empty lists
for i in range(len(memory)):
    if memory[i].text=='Memory':
        HDD.append(memory_specs[i].text.split('/')[0])
        RAM.append(memory_specs[i].text.split('/')[1])
    else:
        HDD.append('Not Available')
        RAM.append('Not Available')

#Extracting the tags having weight

#List of specifications name
weights=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")

#Value of the specifications
weights_specs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")

#Now we will separate weight text from tags
weight=[] #Empty list
for i in range(len(weights)):
    if weights[i].text=='Weight':
        weight.append(weights_specs[i].text)

#Extracting the tags having dimensions

#List of specifications name
dims=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")

#Value of the specifications
dims_specs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")

#Now we will separate dimensions text from tags
dimension=[] #Empty list
for i in range(len(dims)):
    if dims[i].text=='Dimension':
        dimension.append(dims_specs[i].text)

#Extracting the tags having Graphics Processor

#List of specifications name
GPs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")

#Value of the specifications
GPs_specs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")

#Now we will separate GPs text from tags
GPU=[] #Empty list
for i in range(len(GPs)):
    if GPs[i].text=='Graphics Processor':
        GPU.append(GPs_specs[i].text)
        
#Extracting the tags having the price

#As there are some prices in the main url, we will go to full specs and scrape the prices
#First we will extract the urls of all laptop's full specs
full_specs=[]   #Empty list
urls=driver.find_elements_by_xpath("//div[@class='full-specs']/span")

#Running a for loop for extraction of text from tags
for i in urls:
    if i.get_attribute('data-href'):
        full_specs.append(i.get_attribute('data-href'))
        
#Now we will extract price by iterating full_specs
Price=[]  #Empty list
for i in full_specs:
    driver.get(i)
    try:
        prices=driver.find_element_by_xpath("//div[@class='Block-price']/b")  #Getting price tags
        Price.append(prices.text)  #Extracting text
    except NoSuchElementException as e:   #Running an exception if the price is not available
         Price.append("Not Available")    #Message to be printed in places where the price is not available            

In [39]:
#Creating a dataframe for saving our extracted data
laptops=pd.DataFrame({'Product Name':prod_name,'OS':OS,'Display':display_specs,'Processor':processor_specs,'HDD':HDD,'RAM':RAM,
                     'Weight':weight,'Dimension':dimension,'Graphic Processor':GPU,'Price':Price})
laptops

,Product Name,OS,Display,Processor,HDD,RAM,Weight,Dimension,Graphic Processor,Price
0,MSI GT76 TITAN DT 9SG,WINDOWS 10 PRO,"17.3"" (UHD 3840X2160)",INTEL 9TH GEN CORE I9-9900K | 5000 MHZ,1 TB HDD,64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,"379,990"
1,ALIENWARE 17 AREA-51M,WINDOWS 10 PRO,"17.3"" (FHD (1920 X 1080))",INTEL 9TH GEN CORE I9-9900K | 5000 MHZ,1 TB PCIe SSD,32GB DDR4,3.87 Kg,42 mm x 402.6 mm x 319.14 mm,NVIDIA GeForce RTX 2080,"422,000"
2,HP OMEN 15 2020,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",INTEL I7-10750H 10TH GEN | 1.6GHZ,512 GB SSD,16 GBGB DDR4,5.40,14.09 x 9.44 x 0.89,Nvidia GeForce GTX 1650Ti,"117,790"
3,ASUS ZEPHYRUS G14,WINDOWS 10 HOME,"14"" (1920 X 1080)",AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ,1 TB SSD,16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"164,990"
4,LENOVO LEGION Y540,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",9TH GENERATION CORE INTEL I7-9750H | 2.6 GHZ,1 TB SSD,8GB DDR4,2.3,365mm x 260mm x 25.9mm,NVIDIA® GeForce RTX™ 2060,"79,990"
5,ASUS ROG ZEPHYRUS G GA502,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",AMD RYZEN 7 QUAD CORE 3750H | 2.3 GHZ,512 GB SSD,16GB DDR4,2.2,360 x 252 x 20.4,NVIDIA Geforce GTX 1660 Ti,"79,990"
6,ASUS ROG ZEPHYRUS S GX531,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",9TH GEN INTEL CORE I7-8750H | 2.2 GHZ,512GB SSD,16 GBGB DDR4,2.1,360 (W) x 268 (D) x 15.35~16.15 (H) mm,NVIDIA® GeForce RTX™ 2070 (Max-Q),"239,990"
7,MSI GT83VR 7RE TITAN SLI,WINDOWS 10 HOME 64 BIT,"18.4"" (1920 X 1080)",INTEL CM238 CORE I7-7820HK+CM238 7TH GEN | 3.5GHZ,1.5 TB SATA,64GB DDR4,5.5,458 x 339 x 69,Dual GTX1070,"349,990"
8,ASUS ROG ZEPHYRUS DUO 15,WINDOWS 10,"15.6"" (3840 X 1100)",INTEL CORE I7 10TH GEN 10875H | NA,512 GB SSD,4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"244,990"
9,DELL G3,WINDOWS 10 HOME PLUS,15.6 MP | NA,8TH GEN INTEL CORE I5-8300H | 2.3GHZ,1TB HDD,8GB DDR4,2.53,22.7 X 380 X 258,NVidia GeForce GTX 1050,"73,900"


In [40]:
#Closing the driver
driver.close()